### LCEL(LangChain Expression Language )
**长链表达式语言**

#### 概念

- (LangChain Expression Language，简称 LCEL)是一种轻松组合链的声明性方法。

- LCEL 使得从基本组件构建复杂链变得容易，并且支持开箱即用的功能，比如流、并行和日志记录。

> 在新版本中，langchain已经不推荐使用之前的LLMChain了，而是推荐使用LCEL来替代
> [https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain]

#### 优点
【https://python.langchain.com/v0.1/docs/expression_language/】

- 统一的接口 Runnable
- 流式支持
- 异步支持
- 优化的并行执行
- 重试和回退
- 访问中间结果
- 输入和输出模式
- 无缝的LangSmith跟踪集成
- 无缝的LangServe部署集成

In [3]:
# 环境变量设置
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"

### 基础使用 prompt + model + output parser

**LCEL**可以将以上逻辑使用chains串联起来

In [1]:
! pip install -qU langchain-openai

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4", verbose=True, temperature=0)

- 下面代码使用 LCEL 将这些不同的组件拼接成一个链
- | 符号类似于 unix 管道运算符，它将不同的组件链接在一起，将一个组件的输出作为输入提供给下一个组件。

```
chain = prompt | model | output_parser
```

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("请告诉我一个有关于{topic}的笑话")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "冰淇淋"})

"一个男孩走进冰淇淋店，对店员说：“我想要一份香草口味的冰淇淋。”\n\n店员答道：“对不起，我们已经卖完香草口味的了。”\n\n男孩想了想，然后说：“那我要一份香草口味的。”\n\n店员有些没耐心地回答：“我刚才告诉你了，香草口味的卖完了。”\n\n男孩再次想了想，然后说：“那我要一份香草口味的。”\n\n店员无奈地问：“你知道'卖完'是什么意思吗？”\n\n男孩回答：“当然知道，那我要一份香草口味的。”\n\n店员终于忍不住笑出声来：“你真是个坚持的小家伙，那我给你找找看。”"

通过以下步骤将每个步骤串联起来

![alt text](image.png)

### RAG 搜索示例

这个例子将运行一个检索增强生成链，以便在回答问题时添加一些上下文。

In [16]:
! pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 1.3 MB/s eta 0:00:00a 0:00:01m


In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", verbose=True)

In [18]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["小明的父亲在北京工作", "小明喜欢吃西瓜"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("小明的父亲在哪工作?")

'小明的父亲在北京工作。'

组合链使用如下
```
chain = setup_and_retrieval | prompt | model | output_parser
```

链条传递流程如下：
1. 第一个步骤创建一个带有两个条目的 Runnable盲对象。第一个条目，上下文将包括检索器获取的文档结果。第二个条目，问题将包含用户的原始问题。为了传递这个问题，我们使用 RunnablePassthrough 来复制这个条目。

2. 将字典从上面的步骤提供给提示组件。然后，它接受用户输入(即问题)和检索到的文档(即上下文)来构造提示符并输出 PromptValue。

3. 模型组件接受生成的提示符，并传递到 OpenAI LLM 模型进行评估。模型生成的输出是一个 ChatMessage 对象。

4. 最后，output _ parser 组件接受一个 ChatMessage，并将其转换为 Python 字符串，该字符串从调用方法返回。